In [1]:
# Cloud Mask 2.0 

import ee
import yaml
import time
import os
import json
from argparse import ArgumentParser
from utils import clipToROI, exportImageCollectionToGCS, exportImageToGCS, sentinel2CloudScore, calcCloudCoverage
from utils import GEETaskManager

from gevent.fileobject import FileObjectThread

# Polygon Import from Misha ROI List

import pandas as pd

# Functions for Active Run of Cloud Mask 2.0 

from download_sen12 import *

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [18]:
config_file = "config.yml"

In [19]:
stream = open(config_file, 'r') 

In [20]:
config = yaml.load(stream)

<ipython-input-20-8d98dc56653f>:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(stream)


In [ ]:
config["data_list"][0]

# Initialize EE

In [2]:
ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


# Creating New Feature Collection To Use with CM_v2

In [4]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    return polygons

polygons = import_aois("D:/canopy_data/csvs/polygons_101320.csv")

feature_id = 0 
features = []
for poly in polygons[0:3]:
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)

fc = ee.FeatureCollection(features)

In [ ]:
fc

In [ ]:
task_queue = GEETaskManager(n_workers=config['max_tasks'], max_retry=config['max_retry'], wake_on_task=True, log_file=config['log_file'], process_timeout=config['task_timeout'])
task_queue.register_monitor(monitor_tasks)

if os.path.exists('task_log.json'):
    task_log = load_task_log(filename='task_log.json')
    task_queue.set_task_log(task_log)

for data_list in config['data_list'][0:1]:
    for sensor_idx in data_list['sensors']:
        sensor = config['sensors'][sensor_idx]
        print(sensor)
        tasks = process_datasource(task_queue, data_list, sensor, config['export_to'], config['export_dest'], feature_list = fc)

print("Waiting for completion...")
task_queue.wait_till_done()

In [ ]:
d = {'a': 1, 'b': 2, 'c': 3}

d.keys()

In [ ]:
list(d.keys())[0]

In [ ]:
list(d.values())[0]

In [ ]:
d.items()

In [ ]:
e = {'d': d, 'e': 0}

e

In [ ]:
list(list(e.values())[0].keys())[0]

In [ ]:
task_queue = GEETaskManager(n_workers=config['max_tasks'], max_retry=config['max_retry'], wake_on_task=True, log_file=config['log_file'], process_timeout=config['task_timeout'])
task_queue.register_monitor(monitor_tasks)

if os.path.exists('task_log.json'):
    task_log = load_task_log(filename='task_log.json')
    task_queue.set_task_log(task_log)

for data_list in config['data_list']:
    for sensor_idx in data_list['sensors']:
        sensor = config['sensors'][sensor_idx]
        tasks = process_datasource(task_queue, data_list, sensor, config['export_to'], config['export_dest'])

print("Waiting for completion...")
task_queue.wait_till_done()

In [ ]:
!pwd

## makeFilterList

In [ ]:
def makeFilterList(sensor):
    filters_before = None
    filters_after = None

    def _build_filters(filter_list):
        filters = []
        for f in filter_list:
            key = list(f.keys())[0]
            op = list(list(f.values())[0].keys())[0]
            val = list(list(f.values())[0].values())[0]
            filters.append(getattr(ee.Filter, op)(key, val))

        return filters

    if 'filters_before' in sensor:
        filters_before = _build_filters(sensor['filters_before'])

    if 'filters_after' in sensor:
        filters_after = _build_filters(sensor['filters_after'])

    return filters_before, filters_after

In [ ]:
type(config)

In [ ]:
config.keys()

In [ ]:
config

In [ ]:
config['sensors']

In [ ]:
sensor = config['sensors'][0]
sensor

In [ ]:
# Default values
filters_before = None
filters_after = None

In [ ]:
# Sub function
def _build_filters(filter_list):
    # filter_list is a list of dictionaries. Includes the attributes for filtering an image collection
    filters = []
    # for each dict in filter_list
    # example: {'CLOUDY_PERCENTAGE': {'lte': 10}}
    for f in filter_list:
        # key is the first key of the dict -- the feature you're trying to filter by
        # example: 'CLOUDY_PERCENTAGE'
        key = list(f.keys())[0]
        # op is the key of the nested dictionary
        # example: 'lte'
        op = list(list(f.values())[0].keys())[0]
        # val is the value of the nested dictionary
        # example: 10
        val = list(list(f.values())[0].values())[0]
        # Make an ee.Filter object that matches the input filter dict
        # example: ee.Filter.lte('CLOUDY_PERCENTAGE', 10)
        # This will then get applied to an image_collection object
        filters.append(getattr(ee.Filter, op)(key, val))

In [ ]:
help(ee.Filter)

In [ ]:
# Because of the sorting (probably), you may want to apply filters specifically
# before or after creating the image_collection object.
# So we have separate filter lists for both before and after.
if 'filters_before' in sensor:
    filters_before = _build_filters(sensor['filters_before'])

if 'filters_after' in sensor:
    filters_after = _build_filters(sensor['filters_after'])
    
# So at the end, we build a list of ee.Filter objects based off of the sensor
# values for its 'filters_before' and 'filters_after' keys. If the sensor
# lacks one or both such keys, the filters_before and filters_after retain
# their default None value (i.e. no filters get applied).

In [ ]:
ee.Initialize()

In [ ]:
makeFilterList(sensor)

## makeImageCollection

In [ ]:
def makeImageCollection(sensor, roi, start_date, end_date, modifiers=[]):
    # Make the filters based off of the previous function
    filters_before, filters_after = makeFilterList(sensor)

    # Make an image collection. Take the name from the sensor.
    # Filter by date based off of start_date and end_date.
    # Filter bounds based off of the ROI.
    # The map method applies an additional function as a filter; in this case,
    # a clipToROI function that crops every image result in the collection.
    # This way you only have the piece of the image that you're concerned with.
    collection = ee.ImageCollection(sensor['name']) \
                .filterDate(ee.Date(start_date), ee.Date(end_date)) \
                .filterBounds(roi) \
                ### NOTE: Does this need the lambda??
                .map( lambda x: clipToROI(x, ee.Geometry(roi)) )

    # If there are filters_before, apply them
    if filters_before is not None:
        collection = collection.filter( filters_before )

    # If there are additional functions you want to apply, put them in the
    # "modifiers" list and then they will be applied in turn using the 'map' method
    if modifiers and len(modifiers) > 0:
        for m in modifiers:
            collection = collection.map(m)

    # If there are filters_after, apply them
    if filters_after:
        collection = collection.filter( filters_after )

    # 'sensor' states the specific bands you want to take in the 'bands' value.
    # Return those bands of the image collection.
    # This is done at the end just in case other bands are used in custom (pre-)processing--
    # i.e., in the "modifiers" list
    return collection.select(sensor['bands'])

## process_datasource

In [ ]:
def process_datasource(task_queue, source, sensor, export_to, export_dest, feature_list = None):
    # feature_list = ee.FeatureCollection(source['features_src'])
    feature_list = feature_list.sort(source['sort_by']).toList(feature_list.size())
    n_features = feature_list.size().getInfo()

    print("{} features have been loaded".format(n_features))

    task_list = []

    for i in range(1, n_features):
        feature_point = ee.Feature( feature_list.get(i) )

        if source['geometry'] == "point":
            feature_point = feature_point.buffer(source['size']).bounds()

        roi = feature_point.geometry()
        roi = roi.coordinates().getInfo()

        if isinstance(source['name'], str):
            source['name'] = [source['name']]

        if isinstance(sensor['prefix'], str):
            sensor['prefix'] = [sensor['prefix']]

        if 'prefix' in sensor:
            filename_parts = sensor['prefix'] + source['name']
        else:
            filename_parts = source['name']

        filename = "_".join(source['name'] + [str(i)])
        dest_path = "/".join(filename_parts + [filename])

        export_params = {
            'bucket': export_dest,
            'resolution': source['resolution'],
            'filename': filename,
            'dest_path': dest_path
        }

        task_params = {
            'action': export_single_feature,
            'id': "_".join(filename_parts + [str(i)]), # This must be unique per task, to allow to track retries
            'kwargs': {
                'roi': roi,
                'export_params': export_params,
                'sensor': sensor,
                'date_range': {'start_date': source['start_date'], 'end_date': source['end_date']}
            }
        }

        task_queue.add_task(task_params, blocking=True)

In [ ]:
def process_datasource(task_queue, source, sensor, export_to, export_dest, feature_list = None)
### NOTE: We're going to remove the task_queue probably

In [ ]:
# the feature_list is an ee.FeatureCollection
# This sorts the feature_list by the parameter in source['sort_by']
feature_list = feature_list.sort(source['sort_by']).toList(feature_list.size())
# get the number of features in the feature_list
n_features = feature_list.size().getInfo()

In [ ]:
fc

In [ ]:
fc.toList(fc.size())

In [ ]:
help(fc.toList)

In [ ]:
fc.size().getInfo()

In [ ]:
type(fc.size())

In [ ]:
help(ee.ee_number.Number)

In [ ]:
task_list = []
# This variable is not used so I don't know why it's defined

In [ ]:
for i in range(1, n_features):
    feature_point = ee.Feature( feature_list.get(i) )

In [5]:
feature_list = fc.toList(fc.size())

In [ ]:
type(feature_list)

In [ ]:
help(feature_list.get)

In [ ]:
feature_list.get(0).getInfo()

In [ ]:
feature_list.get(1).getInfo()

In [ ]:
## CHANGE:

for i in range(0, n_features):
    # Loop through each feature. Pull out the feature--
    # need to put it inside an "ee.Feature" because otherwise
    # it's a "ComputedObject."
    feature_point = ee.Feature( feature_list.get(i) )

In [6]:
feature_point = ee.Feature( feature_list.get(0) )

In [ ]:
type(feature_list.get(0))

In [ ]:
if source['geometry'] == "point":
    # If the feature is a point, then create a bounding box based off
    # of the "size" attribute from 'source', using the defined
    # feature as the centroid.
    feature_point = feature_point.buffer(source['size']).bounds()

In [7]:
help(feature_point.buffer)

Help on method Feature.buffer in Feature:

Feature.buffer(*args, **kwargs) method of ee.feature.Feature instance
    Returns the input buffered by a given distance. If the distance is
    positive, the geometry is expanded, and if the distance is negative, the
    geometry is contracted.
    
    Args:
      feature: The feature the geometry of which is being buffered.
      distance: The distance of the buffering, which may be
          negative. If no projection is specified, the unit is
          meters. Otherwise the unit is in the coordinate system of
          the projection.
      maxError: The maximum amount of error tolerated when
          approximating the buffering circle and performing any
          necessary reprojection. If unspecified, defaults to 1% of
          the distance.
      proj: If specified, the buffering will be performed in this
          projection and the distance will be interpreted as units of
          the coordinate system of this projection. Otherwis

In [8]:
help(feature_point.bounds)

Help on method Feature.bounds in Feature:

Feature.bounds(*args, **kwargs) method of ee.feature.Feature instance
    Returns a feature containing the bounding box of the geometry of a given
    feature.
    
    Args:
      feature: The feature the bound of which is being calculated.
      maxError: The maximum amount of error tolerated when
          performing any necessary reprojection.
      proj: If specified, the result will be in this projection.
          Otherwise it will be in WGS84.



In [9]:
# Get the coordinates of feature_point as the ROI
roi = feature_point.geometry()
roi = roi.coordinates().getInfo()

In [ ]:
## if type(source['name']) == str
if isinstance(source['name'], str):
    # make it into a list, so we don't error out
    source['name'] = [source['name']]
    
# same as above
if isinstance(sensor['prefix'], str):
    sensor['prefix'] = [sensor['prefix']]

In [ ]:
# make a list 'filename_parts', with all the prefixes
# first (if there are prefixes), then all of the names.
# Keep in mind that at this point, we're working on a single source
# and a single sensor, so really there's just one prefix and one name;
# however, these each might be divided into parts and put into a list
# so that we can then join all the parts together later.
if 'prefix' in sensor:
    filename_parts = sensor['prefix'] + source['name']
else:
    filename_parts = source['name']

In [ ]:
# Filename is the source name, underscore, then an integer
# (integer depends on which feature we're wroking on)
filename = "_".join(source['name'] + [str(i)])
# dest_path is the filename parts joined by backlashses, then the filename
dest_path = "/".join(filename_parts + [filename])

In [ ]:
# Define export parameters
export_params = {
    # export bucket is one of the arguments to the overall function
    'bucket': export_dest,
    # resolution comes from the source
    'resolution': source['resolution'],
    # filename and dest_path defined above
    'filename': filename,
    'dest_path': dest_path
}

In [ ]:
# Define task parameters for the async stuff
task_params = {
    # Function to run: export_single_feature
    'action': export_single_feature,
    # ID for the async stuff to track each task
    'id': "_".join(filename_parts + [str(i)]), # This must be unique per task, to allow to track retries
    'kwargs': {
        # kwargs come from the variables defined in-function
        'roi': roi,
        'export_params': export_params,
        'sensor': sensor,
        'date_range': {'start_date': source['start_date'], 'end_date': source['end_date']}
    }
}

In [ ]:
# async line
task_queue.add_task(task_params, blocking=True)

In [15]:
l1 = [1, 2, 3]
l2 = [4, 5, 6]
l1 + l2

[1, 2, 3, 4, 5, 6]

In [23]:
config['sensors']

[{'name': 'COPERNICUS/S2',
  'prefix': 'S2_CloudFree',
  'type': 'opt',
  'bands': ['B1',
   'B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B8',
   'B8A',
   'B9',
   'B10',
   'B11',
   'B12'],
  'filters_after': [{'CLOUDY_PERCENTAGE': {'lte': 10}}]},
 {'name': 'COPERNICUS/S2',
  'type': 'opt',
  'prefix': 'S2_20_60_Cloud',
  'bands': ['B1',
   'B2',
   'B3',
   'B4',
   'B5',
   'B6',
   'B7',
   'B8',
   'B8A',
   'B9',
   'B10',
   'B11',
   'B12'],
  'filters_after': [{'CLOUDY_PERCENTAGE': {'gte': 20}},
   {'CLOUDY_PERCENTAGE': {'lte': 60}}]},
 {'name': 'COPERNICUS/S1_GRD',
  'type': 'sar',
  'prefix': 'S1',
  'bands': ['VV', 'VH'],
  'filters_before': [{'instrumentMode': {'eq': 'IW'}},
   {'transmitterReceiverPolarisation': {'listContains': 'VV'}},
   {'transmitterReceiverPolarisation': {'listContains': 'VH'}}]}]

In [24]:
sensor = config['sensors'][0]

In [25]:
config.keys()

dict_keys(['downloadDirectory', 'export_to', 'export_dest', 'log_file', 'max_tasks', 'max_retry', 'task_timeout', 'sensors', 'data_list'])

In [26]:
config['data_list']

[{'name': ['summer', '1886'],
  'start_date': '2017-06-01',
  'end_date': '2017-08-31',
  'geometry': 'point',
  'size': 20000,
  'resolution': 10,
  'sort_by': 'name',
  'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
  'sensors': [0, 1, 2]},
 {'name': ['winter', '2017'],
  'start_date': '2016-12-01',
  'end_date': '2017-02-28',
  'geometry': 'point',
  'size': 20000,
  'resolution': 10,
  'sort_by': 'name',
  'features_src': 'ft:1dHi-etD8wtSMPJh-_QT07dKSPO0PE9quSXKCfXeN',
  'sensors': [0, 1, 2]},
 {'name': ['spring', '1158'],
  'start_date': '2017-03-01',
  'end_date': '2017-05-30',
  'geometry': 'point',
  'size': 20000,
  'resolution': 10,
  'sort_by': 'name',
  'features_src': 'ft:1JLPWjSewCd040i_bstDrNrCAvSAaep3i_QRJm-Tb',
  'sensors': [0, 1, 2]},
 {'name': ['autumn', '1970'],
  'start_date': '2017-09-01',
  'end_date': '2017-11-30',
  'geometry': 'point',
  'size': 20000,
  'resolution': 10,
  'sort_by': 'name',
  'features_src': 'ft:1o6ZNS2lkXUiloJ96_UipxR8BWwMo

In [28]:
source = config['data_list'][0]

In [29]:
source['name']

['summer', '1886']

## export_single_feature

In [30]:
def export_single_feature(roi=None, sensor=None, date_range=None, export_params=None):
    modifiers = None
    if sensor['type'].lower() == "opt":
        #print(sensor['type'])
        modifiers = [sentinel2CloudScore, calcCloudCoverage]

    roi_ee = ee.Geometry.Polygon(roi[0])
    image_collection = makeImageCollection(sensor, roi_ee, date_range['start_date'], date_range['end_date'], modifiers=modifiers)
    img = ee.Image(image_collection.mosaic())

    new_params = export_params.copy()
    new_params['img'] = img
    new_params['roi'] = roi

    return exportImageToGCS(**new_params)

In [ ]:
# roi, sensor, date_range, export_params
export_single_feature(roi=None, sensor=None, date_range=None, export_params=None)

In [ ]:
# default modifiers value
modifiers = None
# if the sensor type is "opt" (optical)
if sensor['type'].lower() == "opt":
    #print(sensor['type'])
    # then the modifiers is the following two functions from utils.
    # the only reason to run these functions is if you're getting
    # optical products (i.e. rasters)
    modifiers = [sentinel2CloudScore, calcCloudCoverage]

In [ ]:
# Getting the RoI as an EE Geometry (Polygon) object
roi_ee = ee.Geometry.Polygon(roi[0])

# run the makeImageCollection function that is pulled from utils
image_collection = makeImageCollection(sensor, roi_ee, date_range['start_date'], date_range['end_date'], modifiers=modifiers)

# get a single image by mosaicing the image collection.
# this will naturally do a pixel replacement (i.e. we're flattening the products)
img = ee.Image(image_collection.mosaic())

In [ ]:
# copy the export_params, then add 'img' and 'roi' key/value pairs
new_params = export_params.copy()
new_params['img'] = img
new_params['roi'] = roi

In [ ]:
# run exportImageToGCS (pulled from utils) on the new_params
return exportImageToGCS(**new_params)

In [32]:
help(ee.batch.Export.image)

Help on class image in module ee.batch:

class image(builtins.object)
 |  image(image, description='myExportImageTask', config=None)
 |  
 |  A static class with methods to start image export tasks.
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Forbids class instantiation.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(cls, image, description='myExportImageTask', config=None)
 |      Creates a task to export an EE Image to Google Drive or Cloud Storage.
 |      
 |      Args:
 |        image: The image to be exported.
 |        description: Human-readable name of the task.
 |        config: A dictionary that will be copied and used as parameters
 |            for the task:
 |            - region: The lon,lat coordinates for a LinearRing or Polygon
 |              specifying the region to export. Can be specified as a nested
 |              lists of numbers or a serialized string. Default

In [33]:
help(ee.batch.Export.image.toCloudStorage)

Help on function toCloudStorage in module ee.batch:

toCloudStorage(image, description='myExportImageTask', bucket=None, fileNamePrefix=None, dimensions=None, region=None, scale=None, crs=None, crsTransform=None, maxPixels=None, shardSize=None, fileDimensions=None, skipEmptyTiles=None, fileFormat=None, formatOptions=None, **kwargs)
    Creates a task to export an EE Image to Google Cloud Storage.
    
    Args:
      image: The image to be exported.
      description: Human-readable name of the task.
      bucket: The name of a Cloud Storage bucket for the export.
      fileNamePrefix: Cloud Storage object name prefix for the export.
          Defaults to the name of the task.
      dimensions: The dimensions of the exported image. Takes either a
          single positive integer as the maximum dimension or "WIDTHxHEIGHT"
          where WIDTH and HEIGHT are each positive integers.
      region: The lon,lat coordinates for a LinearRing or Polygon
          specifying the region to expo

# TESTING (ZHENYA START HERE)

In [1]:
import ee
import yaml
import time
import os
import json
import pandas as pd
from utils import exportImageToGDrive
from download_sen12 import *

ee.Initialize()

In [2]:
def process_datasource(source, sensor, export_folder, feature_collection = None):
    # feature_list = ee.FeatureCollection(source['features_src'])
    feature_list = feature_collection.sort(source['sort_by']).toList(feature_collection.size())
    n_features = feature_list.size().getInfo()

    print("{} features have been loaded".format(n_features))

    for i in range(0, n_features):
        feature_point = ee.Feature( feature_list.get(i) )

        if source['geometry'] == "point":
            feature_point = feature_point.buffer(source['size']).bounds()

        roi = feature_point.geometry()
        roi = roi.coordinates().getInfo()

        if isinstance(source['name'], str):
            source['name'] = [source['name']]

        if isinstance(sensor['prefix'], str):
            sensor['prefix'] = [sensor['prefix']]

        if 'prefix' in sensor:
            filename_parts = sensor['prefix'] + source['name']
        else:
            filename_parts = source['name']

        filename = "_".join(source['name'] + [str(i)])
        dest_path = "/".join(filename_parts + [filename])

        export_params = {
            'drive_folder': export_folder,
            'resolution': source['resolution'],
            'filename': filename,
            'dest_path': dest_path
        }
        
        return export_single_feature(roi=roi, export_params=export_params,
                                     sensor=sensor,
                                     date_range={'start_date': source['start_date'],
                                                 'end_date': source['end_date']})
    
    
def export_single_feature(roi=None, sensor=None, date_range=None, export_params=None):
    modifiers = None
    if sensor['type'].lower() == "opt":
        #print(sensor['type'])
        modifiers = [sentinel2CloudScore, calcCloudCoverage]

    roi_ee = ee.Geometry.Polygon(roi[0])
    image_collection = makeImageCollection(sensor, roi_ee, date_range['start_date'], date_range['end_date'], modifiers=modifiers)
    img = ee.Image(image_collection.mosaic())

    new_params = export_params.copy()
    new_params['img'] = img
    new_params['roi'] = roi

    return exportImageToGDrive(**new_params)


def load_config(config_file):
    stream = open(config_file, 'r') 
    return yaml.load(stream)

In [3]:
config_dict = load_config('config.yml')

source = config_dict['data_list'][0]
sensor = config_dict['sensors'][0]
export_folder = config_dict['drive_folder']

<ipython-input-2-091db83a7d82>:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [4]:
def import_aois(csv_loc):    

    df_labels = pd.read_csv(csv_loc)
    df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

    polygons = []
    for polygon in df_labels["polygon"]:
        polygons.append(json.loads(polygon)["coordinates"])

    return polygons

### CHANGE BELOW PATH ###
polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

feature_id = 0 
features = []
for poly in polygons[0:3]:
    # create an roi. first item in Misha's label list
    feature_id += 1 
    
    # create geometry object, create feature object, append to features list for feature collection creation 
    polys = ee.Geometry.Polygon(poly)
    feature = ee.Feature(polys,{"name":feature_id})
    features.append(feature)

fc = ee.FeatureCollection(features)

In [5]:
export = process_datasource(source, sensor, export_folder, fc)

3 features have been loaded


In [14]:
while export.status()["state"] == 'RUNNING':
    print(export.status(), end="\r", flush=True)

In [13]:
help(export)

Help on Task in module ee.batch object:

class Task(builtins.object)
 |  Task(task_id, task_type, state, config=None, name=None)
 |  
 |  A batch task that can be run on the EE batch processing system.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, task_id, task_type, state, config=None, name=None)
 |      Creates a Task with the given ID and configuration.
 |      
 |      The constructor is not for public use. Instances can be obtained by:
 |      - Calling the static method Task.list().
 |      - Calling any of the methods on the Export static class.
 |      - Unpickling a previously pickled Task object.
 |      
 |      If you're looking for a task's status but don't need a full task object,
 |      ee.data.getTaskStatus() may be appropriate.
 |      
 |      Args:
 |        task_id: The task ID, originally obtained through ee.data.newTaskId().
 |          May be None if the ID is not yet known.
 |        task_type: The type of the task; one of the values in Task.Type.
 |  